In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting

#%reload_ext autoreload
#%autoreload 2
%matplotlib inline

import fastai
from fastai.train import Learner
from fastai.train import DataBunch
from fastai.callbacks import *
from fastai.basic_data import DatasetType
import fastprogress
from fastprogress import force_console_behavior
import numpy as np
from pprint import pprint
import pandas as pd
import time


import gc
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold
notebook_start_time = time.time()
tqdm.pandas() # for progress_apply

"""from jig_utils import (JigsawEvaluator,seed_everything, get_coefs, load_embeddings,
                       make_weights_dir, sigmoid, SpatialDropout, save_model_stats,
                       SequenceBucketCollator)"""

from jig_utils import seed_everything

seed_everything(123)

In [ ]:
bert_stacking = np.load('bert_stacking_train.npy')
lstm_stacking = np.load('lstm_stacking_train.npy')

In [ ]:
train_x = bert_stacking[:,0]
train_x = np.c_[train_x, lstm_stacking[:,0]]
train_y = bert_stacking[:,1] # we also need the aux targets!

In [ ]:
DENSE_HIDDEN_UNITS = 8
NUM_DENSE_EPOCHS = 5
NUM_DENSE_MODELS = 2

In [ ]:
class DenseNeuralNet(nn.Module, num_aux_targets):
    def __init__(self):
        super(DenseNeuralNet, self).__init__()
        self.hidden_dropout = Dropout(0.3)
    
        self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linear3 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        
        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)
        self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets)
        
    def forward(self, x, lengths=None):
        x = x.long()
        linear1 = F.relu(self.linear1(x))
        linear1 = self.hidden_dropout(linear1)
        linear2 = F.relu(self.linear2(linear1))
        #SpatialDropout(0.3)
        hidden = F.tanh(self.linear3(linear2 + x))
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        
        out = torch.cat([result, aux_result], 1)

        return out

In [ ]:
def custom_loss(data, targets):
    ''' Define custom loss function for weighted BCE on 'target' column '''
    ### Original code
    bce_loss_1 = nn.BCEWithLogitsLoss(weight=targets[:,1:2])(data[:,:1],targets[:,:1])
    bce_loss_2 = nn.BCEWithLogitsLoss()(data[:,1:],targets[:,2:])
    #bce_loss_1 = nn.MSELoss()(data[:,:1],targets[:,:1])
    #bce_loss_2 = nn.MSELoss()(data[:,1:],targets[:,2:])
    return (bce_loss_1 * loss_weight) + bce_loss_2

In [ ]:
def train_model(learn, model_num,output_dim,lr=0.001,
                batch_size=512, n_epochs=5):
    # use this in inference
    #checkpoint_weights = [1,2,4,8,6]
    n = len(learn.data.train_dl)
    phases = [(TrainingPhase(n).schedule_hp('lr', lr * (0.6**(i)))) for i in range(n_epochs)]
    sched = GeneralScheduler(learn, phases)
    learn.callbacks.append(sched)
    for epoch in range(n_epochs):
        learn.fit(1)

        # saving the model
        model_sin_emb = model.state_dict()
        #learn.save('final_model')
        torch.save(model_sin_emb, "stack_dense" + str(epoch) + "_" + str(model_num))

In [ ]:
batch_size = 512
train_x_torch = torch.tensor(train_x, dtype=torch.float32)
train_y_torch = torch.tensor(train_y, dtype=torch.float32)

train_dataset = data.TensorDataset(train_x_torch, lengths, train_y_torch)
valid_dataset = data.Subset(train_dataset, indices=[0, 1])

train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

databunch = DataBunch(train_dl=train_loader, valid_dl=valid_loader, collate_fn=train_collator)

In [ ]:
for model_idx in range(NUM_DENSE_MODELS):
    print('Model ', model_idx)
    seed_everything(1 + model_idx)
    model = DenseNeuralNet(num_aux_targets=6)
    learn = Learner(databunch, model, loss_func=custom_loss)
    train_model(learn,model_idx,output_dim=7, n_epochs=NUM_DENSE_EPOCHS)